In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split


In [2]:
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
df_train = pd.read_csv('train_GzS76OK/train.csv')
df_fulfilment_center_info = pd.read_csv('train_GzS76OK/fulfilment_center_info.csv')
df_meal_info = pd.read_csv('train_GzS76OK/meal_info.csv')
df_test = pd.read_csv('test_QoiMO9B.csv')

In [4]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [5]:
df_train.describe()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
count,4.565480e+05,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.00000,456548.000000
mean,1.250096e+06,74.768771,82.105796,2024.337458,332.238933,354.156627,0.081152,0.10920,261.872760
std,1.443548e+05,41.524956,45.975046,547.420920,152.939723,160.715914,0.273069,0.31189,395.922798
min,1.000000e+06,1.000000,10.000000,1062.000000,2.970000,55.350000,0.000000,0.00000,13.000000
25%,1.124999e+06,39.000000,43.000000,1558.000000,228.950000,243.500000,0.000000,0.00000,54.000000
50%,1.250184e+06,76.000000,76.000000,1993.000000,296.820000,310.460000,0.000000,0.00000,136.000000
75%,1.375140e+06,111.000000,110.000000,2539.000000,445.230000,458.870000,0.000000,0.00000,324.000000
max,1.499999e+06,145.000000,186.000000,2956.000000,866.270000,866.270000,1.000000,1.00000,24299.000000


In [6]:
df_train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders'],
      dtype='object')

In [7]:
df_fulfilment_center_info.head()

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [8]:
df_fulfilment_center_info.columns

Index(['center_id', 'city_code', 'region_code', 'center_type', 'op_area'], dtype='object')

In [9]:
df_train = pd.merge(df_train, df_fulfilment_center_info, left_on='center_id', right_on='center_id')

In [10]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0


In [11]:
df_train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders', 'city_code',
       'region_code', 'center_type', 'op_area'],
      dtype='object')

In [12]:
df_meal_info.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [13]:
df_train  =  pd.merge(df_train, df_meal_info, left_on='meal_id', right_on='meal_id')

In [14]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1018704,2,55,1885,135.83,152.29,0,0,323,647,56,TYPE_C,2.0,Beverages,Thai
2,1196273,3,55,1885,132.92,133.92,0,0,96,647,56,TYPE_C,2.0,Beverages,Thai
3,1116527,4,55,1885,135.86,134.86,0,0,163,647,56,TYPE_C,2.0,Beverages,Thai
4,1343872,5,55,1885,146.50,147.50,0,0,215,647,56,TYPE_C,2.0,Beverages,Thai


In [15]:
df_train.describe()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,op_area
count,4.565480e+05,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.00000,456548.000000,456548.000000,456548.000000,456548.000000
mean,1.250096e+06,74.768771,82.105796,2024.337458,332.238933,354.156627,0.081152,0.10920,261.872760,601.553399,56.614566,4.083590
std,1.443548e+05,41.524956,45.975046,547.420920,152.939723,160.715914,0.273069,0.31189,395.922798,66.195914,17.641306,1.091686
min,1.000000e+06,1.000000,10.000000,1062.000000,2.970000,55.350000,0.000000,0.00000,13.000000,456.000000,23.000000,0.900000
25%,1.124999e+06,39.000000,43.000000,1558.000000,228.950000,243.500000,0.000000,0.00000,54.000000,553.000000,34.000000,3.600000
50%,1.250184e+06,76.000000,76.000000,1993.000000,296.820000,310.460000,0.000000,0.00000,136.000000,596.000000,56.000000,4.000000
75%,1.375140e+06,111.000000,110.000000,2539.000000,445.230000,458.870000,0.000000,0.00000,324.000000,651.000000,77.000000,4.500000
max,1.499999e+06,145.000000,186.000000,2956.000000,866.270000,866.270000,1.000000,1.00000,24299.000000,713.000000,93.000000,7.000000


In [16]:
df_train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders', 'city_code',
       'region_code', 'center_type', 'op_area', 'category', 'cuisine'],
      dtype='object')

In [17]:
df_test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


In [18]:
df_test = pd.merge(df_test, df_fulfilment_center_info, left_on='center_id', right_on='center_id')

In [19]:
df_test.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'city_code',
       'region_code', 'center_type', 'op_area'],
      dtype='object')

In [20]:
df_test = pd.merge(df_test, df_meal_info, left_on='meal_id', right_on='meal_id')

In [21]:
df_test.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'city_code',
       'region_code', 'center_type', 'op_area', 'category', 'cuisine'],
      dtype='object')

In [22]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1018704,2,55,1885,135.83,152.29,0,0,323,647,56,TYPE_C,2.0,Beverages,Thai
2,1196273,3,55,1885,132.92,133.92,0,0,96,647,56,TYPE_C,2.0,Beverages,Thai
3,1116527,4,55,1885,135.86,134.86,0,0,163,647,56,TYPE_C,2.0,Beverages,Thai
4,1343872,5,55,1885,146.50,147.50,0,0,215,647,56,TYPE_C,2.0,Beverages,Thai


In [26]:
df_train.dtypes

id                         int64
week                       int64
center_id                  int64
meal_id                    int64
checkout_price           float64
base_price               float64
emailer_for_promotion      int64
homepage_featured          int64
num_orders                 int64
city_code                  int64
region_code                int64
center_type               object
op_area                  float64
category                  object
cuisine                   object
city_region               object
dtype: object

In [25]:
# Convert 'city_code' and 'region_code' into a single feature - 'city_region'.
df_train['city_region'] = \
        df_train['city_code'].astype('str') + '_' + \
        df_train['region_code'].astype('str')

df_test['city_region'] = \
        df_test['city_code'].astype('str') + '_' + \
        df_test['region_code'].astype('str')


df_train['city_region'] =  df_train['city_code'].astype('str') + '_' +   df_train['region_code'].astype('str')
df_train['city_region'] =  df_train['city_code'].astype('str') + '_' + df_train['region_code'].astype('str')

In [27]:
# label encoding for categorical features

label_encode_columns = ['center_id', 
                        'meal_id', 
                        'city_code', 
                        'region_code', 
                        'city_region', 
                        'center_type', 
                        'category', 
                        'cuisine']

le = preprocessing.LabelEncoder()

for col in label_encode_columns:
    le.fit(df_train[col])
    df_train[col + '_encoded'] = le.transform(df_train[col])
    df_test[col + '_encoded'] = le.transform(df_test[col])


In [28]:
# Treated week as cyclic feature so convert week into sine and cosine features.

# xsin=sin(2∗π∗xmax(x)) 
# xcos=cos(2∗π∗xmax(x))

In [29]:
df_train['week_sin'] = np.sin(2 * np.pi * df_train['week'] / 52.143)
df_train['week_cos'] = np.cos(2 * np.pi * df_train['week'] / 52.143)

df_test['week_sin'] = np.sin(2 * np.pi * df_test['week'] / 52.143)
df_test['week_cos'] = np.cos(2 * np.pi * df_test['week'] / 52.143)

In [30]:
pd.unique(df_train['cuisine'])

array(['Thai', 'Indian', 'Italian', 'Continental'], dtype=object)

In [31]:
df_train['email_plus_homepage'] =  df_train['emailer_for_promotion'] + df_train['homepage_featured']
df_test['email_plus_homepage'] = df_test['emailer_for_promotion'] +  df_test['homepage_featured']

In [32]:
df_train['num_orders_log1p'] = np.log1p(df_train['num_orders'])


In [33]:
df_train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders', 'city_code',
       'region_code', 'center_type', 'op_area', 'category', 'cuisine',
       'city_region', 'center_id_encoded', 'meal_id_encoded',
       'city_code_encoded', 'region_code_encoded', 'city_region_encoded',
       'center_type_encoded', 'category_encoded', 'cuisine_encoded',
       'week_sin', 'week_cos', 'email_plus_homepage', 'num_orders_log1p'],
      dtype='object')

In [34]:
# columns_to_train = ['week',
#                     'checkout_price',
#                     'base_price',
#                     'email_plus',
#                     'city_region',
#                     'center_type',
#                     'op_area',
#                     'category',
#                     'cuisine',
#                     'center_id',
#                     'meal_id']


columns_to_train = ['week',  'center_id_encoded','meal_id_encoded','checkout_price','week_sin', 'week_cos', 'base_price','city_region_encoded', 'cuisine_encoded', 'category_encoded', 'center_type_encoded', 'op_area']
# columns_to_train = ['week','city_code','region_code', 'op_area']




In [35]:
df_train.dtypes

id                         int64
week                       int64
center_id                  int64
meal_id                    int64
checkout_price           float64
base_price               float64
emailer_for_promotion      int64
homepage_featured          int64
num_orders                 int64
city_code                  int64
region_code                int64
center_type               object
op_area                  float64
category                  object
cuisine                   object
city_region               object
center_id_encoded          int64
meal_id_encoded            int64
city_code_encoded          int64
region_code_encoded        int64
city_region_encoded        int64
center_type_encoded        int64
category_encoded           int64
cuisine_encoded            int64
week_sin                 float64
week_cos                 float64
email_plus_homepage        int64
num_orders_log1p         float64
dtype: object

In [36]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,...,city_code_encoded,region_code_encoded,city_region_encoded,center_type_encoded,category_encoded,cuisine_encoded,week_sin,week_cos,email_plus_homepage,num_orders_log1p
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,...,30,3,30,2,0,3,0.120208,0.992749,0,5.181784
1,1018704,2,55,1885,135.83,152.29,0,0,323,647,...,30,3,30,2,0,3,0.238672,0.971100,0,5.780744
2,1196273,3,55,1885,132.92,133.92,0,0,96,647,...,30,3,30,2,0,3,0.353675,0.935368,0,4.574711
3,1116527,4,55,1885,135.86,134.86,0,0,163,647,...,30,3,30,2,0,3,0.463549,0.886071,0,5.099866
4,1343872,5,55,1885,146.50,147.50,0,0,215,647,...,30,3,30,2,0,3,0.566700,0.823924,0,5.375278


In [37]:
df_train.corr(method='spearman')

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,...,city_code_encoded,region_code_encoded,city_region_encoded,center_type_encoded,category_encoded,cuisine_encoded,week_sin,week_cos,email_plus_homepage,num_orders_log1p
id,1.000000,0.002262,0.002291,0.000365,0.002027,0.002788,0.002001,0.003110,-0.001623,-0.000679,...,-0.000679,-0.000587,-0.000679,0.000479,-0.000291,-0.002638,-0.002397,-0.001292,0.003156,-0.001623
week,0.002262,1.000000,-0.003542,0.018582,0.036299,0.037465,-0.000891,-0.007908,-0.008518,0.000719,...,0.000719,0.004542,0.000719,0.001354,0.004565,-0.017819,-0.173266,-0.135222,-0.014243,-0.008518
center_id,0.002291,-0.003542,1.000000,0.009900,0.001768,-0.000052,0.013817,-0.004174,-0.054553,0.067429,...,0.067429,0.000038,0.067429,-0.109511,-0.010457,-0.021175,0.002715,-0.000259,0.005096,-0.054553
meal_id,0.000365,0.018582,0.009900,1.000000,-0.020744,-0.036978,0.011024,0.014221,0.030271,-0.002070,...,-0.002070,-0.002180,-0.002070,-0.005502,0.216744,0.173656,-0.007375,-0.004070,0.018604,0.030271
checkout_price,0.002027,0.036299,0.001768,-0.020744,1.000000,0.956968,0.015693,-0.058223,-0.388101,-0.005832,...,-0.005832,-0.000636,-0.005832,-0.009517,0.307744,-0.676459,-0.002733,-0.019300,-0.048371,-0.388101
base_price,0.002788,0.037465,-0.000052,-0.036978,0.956968,1.000000,0.150075,0.038812,-0.343826,-0.002585,...,-0.002585,0.004244,-0.002585,-0.008332,0.306227,-0.706868,-0.006483,-0.022020,0.084936,-0.343826
emailer_for_promotion,0.002001,-0.000891,0.013817,0.011024,0.015693,0.150075,1.000000,0.390534,0.211809,-0.004230,...,-0.004230,-0.007755,-0.004230,0.005227,0.092918,-0.150234,0.011583,0.016159,0.739580,0.211809
homepage_featured,0.003110,-0.007908,-0.004174,0.014221,-0.058223,0.038812,0.390534,1.000000,0.232832,0.008927,...,0.008927,0.003423,0.008927,-0.028546,-0.005265,-0.012141,0.027874,0.013926,0.851897,0.232832
num_orders,-0.001623,-0.008518,-0.054553,0.030271,-0.388101,-0.343826,0.211809,0.232832,1.000000,0.025524,...,0.025524,0.028768,0.025524,-0.042609,0.026512,0.193156,-0.010280,0.023074,0.255611,1.000000
city_code,-0.000679,0.000719,0.067429,-0.002070,-0.005832,-0.002585,-0.004230,0.008927,0.025524,1.000000,...,1.000000,0.064484,1.000000,0.064572,0.007747,0.011548,0.000611,0.000331,0.003711,0.025524


In [38]:
X = df_train[columns_to_train]
y = df_train['num_orders_log1p']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, shuffle=False)

In [40]:
estimator = LGBMRegressor(learning_rate=0.003,
                          n_estimators=40000,
                          silent=False)

In [41]:
fit_params = {'early_stopping_rounds': 1000,
              'feature_name': columns_to_train,
#               'categorical_feature': categorical_columns,
              'eval_set': [(X_train, y_train), (X_test, y_test)]}


In [42]:
estimator.fit(X_train, y_train, **fit_params)


[1]	training's l2: 1.44203	valid_1's l2: 1.47156
Training until validation scores don't improve for 1000 rounds
[2]	training's l2: 1.4367	valid_1's l2: 1.46882
[3]	training's l2: 1.4314	valid_1's l2: 1.46611
[4]	training's l2: 1.42613	valid_1's l2: 1.4634
[5]	training's l2: 1.42096	valid_1's l2: 1.46096
[6]	training's l2: 1.41583	valid_1's l2: 1.45852
[7]	training's l2: 1.41072	valid_1's l2: 1.4561
[8]	training's l2: 1.40564	valid_1's l2: 1.45369
[9]	training's l2: 1.4006	valid_1's l2: 1.45129
[10]	training's l2: 1.39559	valid_1's l2: 1.44891
[11]	training's l2: 1.3906	valid_1's l2: 1.44654
[12]	training's l2: 1.38565	valid_1's l2: 1.44418
[13]	training's l2: 1.38072	valid_1's l2: 1.44184
[14]	training's l2: 1.37583	valid_1's l2: 1.4395
[15]	training's l2: 1.37096	valid_1's l2: 1.43718
[16]	training's l2: 1.36612	valid_1's l2: 1.43488
[17]	training's l2: 1.36132	valid_1's l2: 1.43258
[18]	training's l2: 1.35654	valid_1's l2: 1.4303
[19]	training's l2: 1.35179	valid_1's l2: 1.42803
[20]

[163]	training's l2: 0.889796	valid_1's l2: 1.19923
[164]	training's l2: 0.887786	valid_1's l2: 1.198
[165]	training's l2: 0.885655	valid_1's l2: 1.19718
[166]	training's l2: 0.883667	valid_1's l2: 1.19596
[167]	training's l2: 0.881619	valid_1's l2: 1.19494
[168]	training's l2: 0.879521	valid_1's l2: 1.19414
[169]	training's l2: 0.877567	valid_1's l2: 1.19293
[170]	training's l2: 0.875492	valid_1's l2: 1.19213
[171]	training's l2: 0.873557	valid_1's l2: 1.19108
[172]	training's l2: 0.871565	valid_1's l2: 1.19009
[173]	training's l2: 0.869523	valid_1's l2: 1.18931
[174]	training's l2: 0.86763	valid_1's l2: 1.18812
[175]	training's l2: 0.865606	valid_1's l2: 1.18735
[176]	training's l2: 0.863723	valid_1's l2: 1.18632
[177]	training's l2: 0.861787	valid_1's l2: 1.18535
[178]	training's l2: 0.859814	valid_1's l2: 1.18432
[179]	training's l2: 0.857958	valid_1's l2: 1.18358
[180]	training's l2: 0.8561	valid_1's l2: 1.18284
[181]	training's l2: 0.854157	valid_1's l2: 1.18182
[182]	training's 

[331]	training's l2: 0.652658	valid_1's l2: 1.04098
[332]	training's l2: 0.651739	valid_1's l2: 1.04009
[333]	training's l2: 0.650834	valid_1's l2: 1.0392
[334]	training's l2: 0.649918	valid_1's l2: 1.03789
[335]	training's l2: 0.64905	valid_1's l2: 1.03736
[336]	training's l2: 0.648154	valid_1's l2: 1.03664
[337]	training's l2: 0.64722	valid_1's l2: 1.03534
[338]	training's l2: 0.646343	valid_1's l2: 1.03492
[339]	training's l2: 0.645464	valid_1's l2: 1.03405
[340]	training's l2: 0.644574	valid_1's l2: 1.03277
[341]	training's l2: 0.643733	valid_1's l2: 1.03225
[342]	training's l2: 0.642875	valid_1's l2: 1.03055
[343]	training's l2: 0.642018	valid_1's l2: 1.03046
[344]	training's l2: 0.641161	valid_1's l2: 1.0296
[345]	training's l2: 0.640302	valid_1's l2: 1.02803
[346]	training's l2: 0.63945	valid_1's l2: 1.02747
[347]	training's l2: 0.638599	valid_1's l2: 1.0259
[348]	training's l2: 0.637787	valid_1's l2: 1.0254
[349]	training's l2: 0.636959	valid_1's l2: 1.02555
[350]	training's l2

[499]	training's l2: 0.543423	valid_1's l2: 0.927348
[500]	training's l2: 0.542981	valid_1's l2: 0.926459
[501]	training's l2: 0.542511	valid_1's l2: 0.926296
[502]	training's l2: 0.542052	valid_1's l2: 0.926207
[503]	training's l2: 0.541593	valid_1's l2: 0.926198
[504]	training's l2: 0.541165	valid_1's l2: 0.92571
[505]	training's l2: 0.540704	valid_1's l2: 0.925543
[506]	training's l2: 0.540245	valid_1's l2: 0.925378
[507]	training's l2: 0.539835	valid_1's l2: 0.924536
[508]	training's l2: 0.539357	valid_1's l2: 0.924511
[509]	training's l2: 0.538919	valid_1's l2: 0.924585
[510]	training's l2: 0.538446	valid_1's l2: 0.924561
[511]	training's l2: 0.538021	valid_1's l2: 0.924036
[512]	training's l2: 0.537569	valid_1's l2: 0.923874
[513]	training's l2: 0.537111	valid_1's l2: 0.923945
[514]	training's l2: 0.536692	valid_1's l2: 0.923424
[515]	training's l2: 0.536247	valid_1's l2: 0.92326
[516]	training's l2: 0.535794	valid_1's l2: 0.923244
[517]	training's l2: 0.535381	valid_1's l2: 0.92

[666]	training's l2: 0.484053	valid_1's l2: 0.864603
[667]	training's l2: 0.483824	valid_1's l2: 0.864537
[668]	training's l2: 0.483537	valid_1's l2: 0.863833
[669]	training's l2: 0.483277	valid_1's l2: 0.863621
[670]	training's l2: 0.483023	valid_1's l2: 0.862994
[671]	training's l2: 0.482754	valid_1's l2: 0.862616
[672]	training's l2: 0.482503	valid_1's l2: 0.862319
[673]	training's l2: 0.482231	valid_1's l2: 0.861678
[674]	training's l2: 0.481979	valid_1's l2: 0.861351
[675]	training's l2: 0.481732	valid_1's l2: 0.861059
[676]	training's l2: 0.481458	valid_1's l2: 0.86055
[677]	training's l2: 0.481202	valid_1's l2: 0.861114
[678]	training's l2: 0.480922	valid_1's l2: 0.860432
[679]	training's l2: 0.480669	valid_1's l2: 0.860242
[680]	training's l2: 0.480453	valid_1's l2: 0.86026
[681]	training's l2: 0.480189	valid_1's l2: 0.859631
[682]	training's l2: 0.479937	valid_1's l2: 0.860193
[683]	training's l2: 0.479675	valid_1's l2: 0.859826
[684]	training's l2: 0.479427	valid_1's l2: 0.85

...

[39381]	training's l2: 0.200229	valid_1's l2: 0.594195
[39382]	training's l2: 0.200228	valid_1's l2: 0.594191
[39383]	training's l2: 0.200227	valid_1's l2: 0.59418
[39384]	training's l2: 0.200226	valid_1's l2: 0.594181
[39385]	training's l2: 0.200225	valid_1's l2: 0.594181
[39386]	training's l2: 0.200223	valid_1's l2: 0.594177
[39387]	training's l2: 0.200221	valid_1's l2: 0.594175
[39388]	training's l2: 0.20022	valid_1's l2: 0.594176
[39389]	training's l2: 0.200219	valid_1's l2: 0.594168
[39390]	training's l2: 0.200218	valid_1's l2: 0.594176
[39391]	training's l2: 0.200217	valid_1's l2: 0.594176
[39392]	training's l2: 0.200217	valid_1's l2: 0.594173
[39393]	training's l2: 0.200215	valid_1's l2: 0.594174
[39394]	training's l2: 0.200215	valid_1's l2: 0.594174
[39395]	training's l2: 0.200214	valid_1's l2: 0.594173
[39396]	training's l2: 0.200213	valid_1's l2: 0.594165
[39397]	training's l2: 0.200212	valid_1's l2: 0.594166
[39398]	training's l2: 0.200211	valid_1's l2: 0.594126
[39399]	trai

[39538]	training's l2: 0.20008	valid_1's l2: 0.594006
[39539]	training's l2: 0.200078	valid_1's l2: 0.594007
[39540]	training's l2: 0.200078	valid_1's l2: 0.594008
[39541]	training's l2: 0.200077	valid_1's l2: 0.594007
[39542]	training's l2: 0.200076	valid_1's l2: 0.594008
[39543]	training's l2: 0.200076	valid_1's l2: 0.594001
[39544]	training's l2: 0.200075	valid_1's l2: 0.594001
[39545]	training's l2: 0.200075	valid_1's l2: 0.594002
[39546]	training's l2: 0.200074	valid_1's l2: 0.594002
[39547]	training's l2: 0.200074	valid_1's l2: 0.594003
[39548]	training's l2: 0.200073	valid_1's l2: 0.594002
[39549]	training's l2: 0.200073	valid_1's l2: 0.594002
[39550]	training's l2: 0.200072	valid_1's l2: 0.594004
[39551]	training's l2: 0.200071	valid_1's l2: 0.594003
[39552]	training's l2: 0.20007	valid_1's l2: 0.594003
[39553]	training's l2: 0.20007	valid_1's l2: 0.594003
[39554]	training's l2: 0.200069	valid_1's l2: 0.594003
[39555]	training's l2: 0.200068	valid_1's l2: 0.594003
[39556]	train

[39696]	training's l2: 0.199943	valid_1's l2: 0.5938
[39697]	training's l2: 0.199942	valid_1's l2: 0.593799
[39698]	training's l2: 0.199942	valid_1's l2: 0.593799
[39699]	training's l2: 0.199941	valid_1's l2: 0.593799
[39700]	training's l2: 0.199941	valid_1's l2: 0.593796
[39701]	training's l2: 0.19994	valid_1's l2: 0.593795
[39702]	training's l2: 0.199939	valid_1's l2: 0.593795
[39703]	training's l2: 0.199939	valid_1's l2: 0.593795
[39704]	training's l2: 0.199939	valid_1's l2: 0.593795
[39705]	training's l2: 0.199937	valid_1's l2: 0.593774
[39706]	training's l2: 0.199936	valid_1's l2: 0.593773
[39707]	training's l2: 0.199936	valid_1's l2: 0.593773
[39708]	training's l2: 0.199935	valid_1's l2: 0.593774
[39709]	training's l2: 0.199935	valid_1's l2: 0.593774
[39710]	training's l2: 0.199934	valid_1's l2: 0.593774
[39711]	training's l2: 0.199933	valid_1's l2: 0.59377
[39712]	training's l2: 0.199933	valid_1's l2: 0.59377
[39713]	training's l2: 0.199931	valid_1's l2: 0.59377
[39714]	training

[39853]	training's l2: 0.199776	valid_1's l2: 0.593702
[39854]	training's l2: 0.199776	valid_1's l2: 0.593702
[39855]	training's l2: 0.199775	valid_1's l2: 0.593701
[39856]	training's l2: 0.199773	valid_1's l2: 0.593712
[39857]	training's l2: 0.199772	valid_1's l2: 0.593712
[39858]	training's l2: 0.199771	valid_1's l2: 0.593713
[39859]	training's l2: 0.19977	valid_1's l2: 0.593712
[39860]	training's l2: 0.199768	valid_1's l2: 0.593712
[39861]	training's l2: 0.199768	valid_1's l2: 0.593712
[39862]	training's l2: 0.199766	valid_1's l2: 0.593713
[39863]	training's l2: 0.199765	valid_1's l2: 0.593714
[39864]	training's l2: 0.199765	valid_1's l2: 0.593713
[39865]	training's l2: 0.199763	valid_1's l2: 0.593707
[39866]	training's l2: 0.199762	valid_1's l2: 0.593707
[39867]	training's l2: 0.19976	valid_1's l2: 0.593699
[39868]	training's l2: 0.19976	valid_1's l2: 0.593699
[39869]	training's l2: 0.199759	valid_1's l2: 0.593698
[39870]	training's l2: 0.199757	valid_1's l2: 0.593696
[39871]	train

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.003, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=40000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [43]:
df_test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,...,meal_id_encoded,city_code_encoded,region_code_encoded,city_region_encoded,center_type_encoded,category_encoded,cuisine_encoded,week_sin,week_cos,email_plus_homepage
0,1028232,146,55,1885,158.11,159.11,0,0,647,56,...,22,30,3,30,2,0,3,-0.951071,0.308971,0
1,1262649,147,55,1885,159.11,159.11,0,0,647,56,...,22,30,3,30,2,0,3,-0.907034,0.421057,0
2,1453211,149,55,1885,157.14,158.14,0,0,647,56,...,22,30,3,30,2,0,3,-0.780327,0.625372,0
3,1262599,150,55,1885,159.14,157.14,0,0,647,56,...,22,30,3,30,2,0,3,-0.699494,0.714639,0
4,1495848,151,55,1885,160.11,159.11,0,0,647,56,...,22,30,3,30,2,0,3,-0.608516,0.793541,0


In [44]:
X = df_test[columns_to_train]


In [45]:
pred = estimator.predict(X)
pred = np.expm1(pred)


In [48]:
submission_df = df_test.copy()
submission_df['num_orders'] = pred
submission_df = submission_df[['id', 'num_orders']]
submission_df.to_csv('submission.csv', index=False)